<a href="https://colab.research.google.com/github/yuukimiyo/CanvasWater/blob/master/sample_topic_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

LSA（潜在的意味理解）を用いて文章からTopic抽出する処理のサンプルプログラムです。


日本語コーパスとして無料公開されているLivedoorニュースコーパスの個別カテゴリの全記事から、指定した数のトピック（話題）を抽出する処理を行います。


【技術解説】<br />
*   Livedoorニュースコーパス：Livedoorニュースの記事から公開に差し障りのある部分を削除し、一般公開されている日本語コーパスです
*   潜在意味解析(LSA) ：「～特異値分解(SVD)から文書検索まで～( https://mieruca-ai.com/ai/lsa-lsi-svd/ )」
*   コーパス：広義では分析に用いる文章セットの事をさす。プログラム内で使用された場合は特定のデータ構造を言う場合が多いので他人のコードを読む際は注意が必要です。




# 使い方

コードが記載された各セル（背景が灰色）をクリックし、[Shift]キー+[Enter]キーでセル内の処理を実行してください。処理が実行されると処理対象が次のセルに自動的に遷移するので、上記キーを繰り返し押して処理を進めてください。

# 謝辞

このサンプルプログラムではクリエイティブコモンズライセンスの元に公開されているLivedoor-corpusを利用しています。<br />
https://www.rondhuit.com/download.html

貴重な日本語テキストコーパスを公開して頂いたNHN Japan株式会社およびプログラム中で使用している各種ライブラリ等の製作者に謝意を表します。

# 前処理


---

Loggerの初期化、MeCabのインストールとLivedoorニュースコーパスのダウンロードなどを行います。<br />
少々時間がかかるので時間に余裕のあるときに実行してください。

### Loggerを初期化

各種ライブラリから出力されるLog情報を扱うためのLoggerをセットアップします。

In [0]:
# Loggerを初期化
# Colaboratory上でloglevelを変更した場合、ランタイムの再起動が必要
from logging import basicConfig, getLogger, INFO as LOGLEVEL
basicConfig(level=LOGLEVEL, format='%(asctime)s(%(levelname)s) %(name)s: %(message)s', datefmt='%I:%M:%S')
logger = getLogger(__name__)

### MeCab＋Neologd をダウンロード＆インストール

この処理には3～4分ほどの時間がかかります。

In [0]:
# MeCab + Neologd辞書のインストール
# 処理に３～４分かかります！
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab swig
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

# pipを実行
!pip install mecab-python3

### Livedoor-corpusをダウンロード
Livedoorcorpusをダウンロードして解凍します。
1分ほどの時間がかかります。

In [0]:
# Livedoorコーパスのダウンロード
!wget http://www.rondhuit.com/download/ldcc-20140209.tar.gz
    
 # Livedoorコーパスの解凍
!tar xzf ldcc-20140209.tar.gz

In [0]:
# 解凍結果を確認します。
# dokujo-tsushinなどのLivedoorcorpusのカテゴリ名が並んでいれば成功です
!ls -l ./text

### その他前処理

In [0]:
# gensimモジュールのimport時に要求されるモジュールを予めインストールします。
!pip install pattern

# メイン処理


---



### ライブラリの読み込みと定数定義

In [0]:
# 利用ライブラリの読み込み
import os
import glob
import csv
import codecs
import re

import MeCab
import pandas as pd

from gensim import corpora, models, similarities

# システム辞書としてNeologdを指定してMeCabをセットアップ
tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati -E""')

# 入力するテキストの文字コードを指定
input_charset = "utf-8"

### 処理対象のカテゴリを指定

Livedoorニュースコーパスに含まれるカテゴリから一つ選んで指定します。<br />
今回のサンプルでは独女通信を選んで指定していますが、もちろん変更しても構いません。
 - dokujo-tsushin (独女通信)
 - it-life-hack (ITライフハック)
 - kaden-channel (家電チャンネル)
 - livedoor-homme (livedoor HOMME)
 - movie-enter (MOVIE ENTER)
 - peachy (Peachy)
 - smax (エスマックス)
 - sports-watch (Sports Watch)
 - topic-news (トピックニュース)
 
 


In [0]:
# 読み込み対象のカテゴリ名を指定
category_name = "movie-enter"

### クリーン処理用関数の作成

配布されているLivedoorコーパスには、日付や文字としての[◆]、URLなど分析の際にノイズとなる文字列が多く含まれています。<br />
そういったノイズを取り除くための関数を作成します。

In [0]:
# クリーン関数を作成

# 正規表現置き換えのためのパターンをコンパイルし、配列に格納する。
# ここでいうコンパイルはC言語などのコンパイルとは違い、正規表現処理などを高速に行うためのPython独自のテクニックのこと
ptns = []
ptns.append(re.compile(r"@pad_sexy"))
ptns.append(re.compile(r"\\+n")) # 改行
ptns.append(re.compile(r"[\s]+")) # スペース文字
ptns.append(re.compile(r"\([^\(]{1,5}\)")) # 顔文字
ptns.append(re.compile(r"#[!-~]+")) # 「# + 記号」の記述を削除
ptns.append(re.compile(r"@[!-~]+")) # 「@ + 記号」の記述を削除
ptns.append(re.compile(r"[○|●|□|■|△|▲|▽|▼|◆|◇|【|】|※|★|☆]"))
ptns.append(re.compile(r'(https?|ftp)(:\/\/[-_.!~*\'()a-zA-Z0-9;\/?:\@&=+\$,%#]+)')) # URLを削除
ptns.append(re.compile(r'(\d{4})-(\d{2})-(\d{2})[T|t](\d{2}):(\d{2}):(\d{2})\+(\d{4})')) # システム形式の日付表現を削除

# 全角->半角変換（のための変換表をあらかじめコンパイルしておく）
zen_chars = "".join(chr(0xff01 + i) for i in range(94))
han_chars = "".join(chr(0x21 + i) for i in range(94))
zen2han = str.maketrans(zen_chars, han_chars)


def cleanText(t):
    """入力された文字列にクリーン処理を実施して結果を返す
    Args:
        t (str): 元の文字列
    Returns:
        str: 整形後の文字列
    """
    
    # 前後の改行等削除
    t = t.strip()
    
    # 全角を半角に
    t = t.translate(zen2han)
    
    # アルファベットをすべて小文字に
    t = t.lower()
    
    # 正規表現を用いて、分析の邪魔になる部分を削除する。
    # 配列に格納された正規表現マッチの処理を逐次適用して文字列削除を行う。
    for ptn in ptns:
        t = ptn.sub("", t)
    
    return t

### Livedoorコーパスを読み込み、クリーン処理および分かち書きを実施

入力対象のフォルダにはLivedoorコーパスとして、ブログの１つの記事が１つのファイルとして格納されています。
各記事のテキストに対してクリーン処理を実施し、一つの記事が１行となる配列を作成します。

In [0]:
# 入力するLivedoorcorpusの記事テキストが格納されたフォルダへのパス
input_dir = os.path.join(".", "text", category_name)

# 読み込み対象のファイル一覧を取得
all_files = glob.glob("{}/*.txt".format(input_dir))

# 一つずつファイルを読み込んで、それぞれのファイルについて一行ずつでクリーン処理を実施します
# 今回は全て行を一旦配列に格納していますが、入力が膨大になる場合はメモリ効率を考慮した処理に変更する必要があります。

input_lines = []
for i, file_path in enumerate(all_files):
    # ファイルリストから、個別のファイル名を取得し、ひとつづつ処理する
    with codecs.open(file_path, 'r', encoding=input_charset) as f:
        each_file_lines = []
        for l in f.readlines():
            l = cleanText(l)
            if len(l) <= 3:
                next;
            
            each_file_lines.append(l)
            
        # ファイル内の全行を読み込んだら、一行にして分析用の配列（input_lines）に追加します
        input_lines.append("".join(each_file_lines))

### クリーン処理の結果に対して分かち書きを実施

In [0]:
# 分かち書きを実施
input_lines_wakati = []
for l in input_lines:
    
    # 分かち書きを実施
    l = tagger.parse(l)
    
    # 単語数が3つ以上の場合に、分析用配列(input_lines_wakati)に追加します。
    if len(l) >= 3: # 単語数が3つ以下の記事があった場合はここで足切りする
        input_lines_wakati.append(l.split(" "))

### 分かち書きの結果を表示
確認のため、単語IDの配列の一部を表示します。<br />
（この部分は実行しなくても問題ないです）<br />

In [0]:
# 変換結果の確認
for i, line in enumerate(input_lines_wakati):
    print(line)
    if i > 10:
        break;

## 辞書の作成

単語配列を単語ID配列に変換するための辞書を作成するします。

In [0]:
# 単語配列化したTweetを辞書化（単語番号に変換）するための辞書を作成
dictionary = corpora.Dictionary(input_lines_wakati)

## 単語ID配列への変換
分かち書きの結果を、単語IDの配列に変換します。

In [0]:
# 上記辞書を用いて単語配列を単語番号配列に変換（この単語番号配列をCorpusと呼ぶ場合が多い）
corpus = [dictionary.doc2bow(line) for line in input_lines_wakati]

### 変換結果の確認
確認のため、単語IDの配列の一部を表示します。<br />
（この部分は実行しなくても問題ないです）<br />
<br />
配列の中身は、(単語ID, 記事内での単語の出現数)となっています。

In [0]:
# 変換結果の確認
for i, row in enumerate(corpus):
    print(row)
    if i > 10:
        break;

### 辞書の確認
先の手順で作成した辞書オブジェクトは配列として単語IDを与えるとそのIDが指し示している単語を返します。<br />
興味のある方は次のように単語IDを与えて単語を表示してみましょう。
（この部分は実行しなくても問題ないです）

In [0]:
dictionary[40]

## 各記事をベクトル化する

どれだけ珍しい単語を使っているかを手掛かりにして、各記事のベクトル表現を得ます。

（ここでのベクトルはスパースベクトルとして表現されています。興味のある方は「スパースベクトル」で検索してみてください）

In [0]:
# 単語ID配列を用い、Tfidf重みを使ってベクトル化

# tfidf重みを算出するためのモデルファイルを作成
tfidf = models.TfidfModel(corpus)

# モデルファイルを元に、ベクトル化を実施
corpus_tfidf = tfidf[corpus]

### ベクトル化した結果を表示
各単語IDに対して、tfidfによって算出された重みが付与されたデータが作成されています。<br />
（この部分は実行しなくても問題ないです）

In [0]:
# ベクトル化した結果を表示
for i, row in enumerate(corpus_tfidf):
    print(row)
    if i > 10:
        break;

## トピック抽出を実行
LSI(潜在的意味解析)を用いてトピックを抽出する処理です。<br />
gensimというライブラリのLsiModelという関数を使用してトピックオブジェクトを作成しています。<br />
関数の使い方は次の公式URLに詳しく書いてあります。<br />
https://radimrehurek.com/gensim/models/lsimodel.html

In [0]:
# lsaによるトピックの抽出を実行
lsi = models.LsiModel(corpus=corpus_tfidf, id2word=dictionary,
                             num_topics=20, chunksize=10000)

In [0]:
lsi = models.LsiModel(corpus=corpus_tfidf, num_topics=20, chunksize=10000)

## トピックの確認
トピックオブジェクトから、show_topic()関数を用いて個別のトピックを抽出して表示しています。

In [0]:
# トピックリストを作成
for i in range(len(lsi.get_topics())):
    each_topic_words = []
    for w in lsi.show_topic(i):
        each_topic_words.append("{}:{:.4f}".format(dictionary[int(w[0])], w[1]))
    print("topic {}: {}".format(i, ", ".join(each_topic_words)))

以上でLivedoorニュースコーパスの特定カテゴリから、指定した数のトピック（話題）を抽出する処理は完了です。<br />
次からは抽出したトピックから一つの話題を選び、その話題に近い記事を抽出する、別の処理を作成します。

# 個別のトピックに関連する記事を抽出する


---

以降の処理はトピック分析だけでなく、文章ベクトルなど大量のベクトルから特定の特徴を表したベクトルを抽出する目的で汎用的に使用できる処理です。<br />
例えば、特定の単語を含んだニュースを抽出するなどの目的にも使用することができます。

### 対象とするトピックを選択

In [0]:
# トピックリストから、対象とするトピックをトピック番号を指定して抽出
target_topic = lsi.show_topic(7)

### 対象とするトピックを表示
確認のため単語IDを単語に直して表示してみます<br />
トピック分析などで使用する際は、ここで表示するトピックベクトルに該当するデータを自分で作成すればよいわけです。

In [0]:
# 選択したトピックを確認
[(dictionary[int(t[0])], t[1]) for t in target_topic]

### トピック関連記事の抽出
全記事を対象に、トピックとの類似度（ベクトルのコサイン距離）を算出し、類似度リストを作成します。

In [0]:
# 全記事を対象に、トピックベクトルとのコサイン距離を取得する
similarity_index = similarities.SparseMatrixSimilarity(corpus_tfidf, num_features = len(dictionary))
similarity_scores = list(similarity_index[target_topic])

### 類似度リストを個別の記事に関連付ける
類似度リストは記事と同じ順序でトピックベクトルとの距離が格納されている配列なので、本文データと紐づけた配列を作成して降順にソートします。

In [0]:
# トピックとのコサイン距離、記事番号、記事本文、によるリストを作成する
topic_scores = []
for i, line in enumerate(input_lines):
    topic_scores.append([similarity_scores[i], i, line])

# 上記リストをコサイン距離の大きい順に並べ替える
topic_scores = sorted(topic_scores, reverse=True)

### 抽出結果を表示
トピックとの関連度順なっているデータの先頭50件を表示します。

In [0]:
for i, t in enumerate(topic_scores):
    print(t)
    if i > 50:
        break